<h1> Comparison of City vs. National Climate Change Mitigation Policies: Data Processing & Compiling </h1>

This is a script I've used on a project where I needed to gather data on the greenhouse gas emissions, climate change mitigation actions, emissions reduction targets and mitigation policies for cities and their respective countries. I gathered, cleaned and aggregated the data for four city-country pairs as case studies. These were then exported to separate Excel files as 'factsheets' for further analysis.

City data on the following aspects from <b><i> CDP's 2022 Cities Questionnaire Response Data </i></b> (publicly available [via their data portal](https://data.cdp.net/browse))
- _City emissions inventories_
- _City emissions reduction actions/measures_
- _Drafted city mitigation plans_
- _City emissions reduction targets_
- _City population (reported within the previously listed datasets)_

National data on the following aspects and from the following sources:
- _National emissions inventories_ (CAIT data taken from [Climate Watch platform](https://www.climatewatchdata.org/ghg-emissions?end_year=2019&start_year=1990); separate files downloaded for each country needed)
- _National emissions reduction targets_ (manually compiled into a csv file based on info from the [Climate Action Tracker](https://climateactiontracker.org/))
- _National climate change policies_ (dataset downloaded from New Climate Institute's [Climate Policy Database](https://climatepolicydatabase.org/))
- _National population_ (dataset downloaded from the [World Bank](https://data.worldbank.org/indicator/SP.POP.TOTL))

In [1]:
import numpy as np
import pandas as pd

## City data files to be imported; downloaded from CDP's 2022 Cities Questionnaire Data as separated datasets:
city_files = ['20220424_Cities_Measures','20220424_Cities_Targets', '20220424_Cities_Emissions', '20220424_Cities_Mitigation_Plans']

dir_data = 'City_Country_Data/'

city_frames = []

for i in range(len(city_files)):
    ## Read each file of city data
    c_df = pd.read_csv(r''+dir_data+city_files[i]+'.csv', on_bad_lines='skip', sep=',')
    ## Drop these unnecessary columns from each dataset
    c_df.drop(['CDP Region', 'Access', 'C40 City', 'GCoM City'], axis=1, inplace=True)
    ## Append to empty list for further processing
    city_frames.append(c_df)
    
city_mitactions = city_frames[0]
city_targets = city_frames[1]
city_emissions = city_frames[2]
city_mitplans = city_frames[3]

<h1> City Data: Cleaning </h1>

<h2> Emissions Data </h2>

In [2]:
## Check data available in the dataset
city_emissions.columns

Index(['Questionnaire', 'Organization Number', 'Organization Name', 'City',
       'Country', 'Number of times reporting',
       'Emissions reporting framework or protocol', 'Gases Included',
       'Emissions Question Number', 'Emissions Question Name',
       'Emissions Column Number', 'Emissions Column Name',
       'Emissions Row Number', 'Emissions Row Name',
       'Emissions Response Answer', 'Emissions Notation Key',
       'Emissions Description', 'Emissions Data Reported',
       'Emissions Data Group', 'Emissions Rank', 'Inventory Year',
       'Inventory boundary (relative to jurisdiction boundary)',
       'Population in inventory year', 'Tool used to compile inventory',
       'City_Location', 'Last update'],
      dtype='object')

In [3]:
## Dropping remaining unnecessary columns
city_emissions.drop(['Emissions Question Number','Emissions Question Name','Emissions Column Number', 'Emissions Notation Key', 'Emissions Data Reported', 'Emissions Rank'], axis=1, inplace=True)

## Sort by emissions category
city_emissions.sort_values(by=['Emissions Row Number'],  inplace=True)

## Filter our this emissions category
city_emissions = city_emissions[city_emissions['Emissions Column Name'] != 'Emissions occurring outside the jurisdiction boundary as a result of in-jurisdiction activities (metric tonnes CO2e)']

## Filter for certain types of emissions data groups: sector and source totals for each category, as well as any category related to energy or buildings
city_emissions = city_emissions[((city_emissions['Emissions Data Group'] == 'SectorTotal') | (city_emissions['Emissions Data Group'] == 'SourceTotal') |
(city_emissions['Emissions Data Group'].str.contains('grid', case=False))) | (city_emissions['Emissions Description'].str.contains('building', case=False))]



In [4]:
## Need to clean: Emissions were entered in the data set as a text field with commas as thousands separator
## Remove commas
city_emissions['Emissions Response Answer'] = city_emissions['Emissions Response Answer'].str.replace(',','', regex=False)
## Convert to numerical data type
city_emissions['Emissions Response Answer'].astype('float64', copy=False)

37321          0.00
9304       83879.00
3552       94519.89
1203      157565.00
34020      54337.00
            ...    
18247     119850.57
38161    2269512.00
37200    1968753.00
12077    9574000.00
2421       72094.00
Name: Emissions Response Answer, Length: 11999, dtype: float64

<h2> Emissions Reduction Actions </h2>

In [5]:
## Check available data
city_mitactions.columns

Index(['Questionnaire', 'Organization Number', 'Organization Name', 'City',
       'Country', 'Number of times reporting', 'Primary emissions sector',
       'Action type', 'Action description and web link',
       'Start year of action', 'Year for which mitigation is expected to last',
       'Impact indicators measured',
       'Estimated annual emissions reductions (metric tons CO2e/year)',
       'Estimated annual energy savings (MWh/year)',
       'Estimated annual renewable energy generation (MWh/year)',
       'Co-benefits realised', 'Funding source(s)',
       'Status of action in the reporting year',
       'Inclusion in climate action plan and/or jurisdiction development / master plan',
       'Total cost of action', 'Currency', 'Population', 'Population Year',
       'City Location', 'Last update'],
      dtype='object')

In [6]:
## Check available sector for emissions reduction actions
city_mitactions['Primary emissions sector'].value_counts()

Stationary energy                     1471
Transportation                        1169
Waste                                  625
Agriculture, Forestry and Land Use     251
Other sectoral action                  249
Generation of grid-supplied energy     203
Other, please specify                  110
Food                                    71
Industrial Processes and Product        47
No mitigation action in place            6
Name: Primary emissions sector, dtype: int64

We can see here the economic sectors into which emissions reduction actions are categorized. For my work, I was interested in measures related to energy, waste, transportation and buildings. However, we can see here that buildings is not disaggregated as a separate category. Generally in emissions inventories, buidlings is a subsector of energy. Below I check the subcategories for rows where Stationary Energy is listed as the primary emissions sector.

In [7]:
## Check the most common action types for energy sector emissions reduction actions
energy_action_types = pd.DataFrame(city_mitactions[city_mitactions['Primary emissions sector'] == 'Stationary energy']['Action type'].value_counts())

energy_action_types.head(20)

,Action type
"Energy efficiency/ retrofit measures addressing existing commercial, residential and/or municipal buildings",505
On-site renewable energy generation,260
LED / CFL / other luminaire technologies,133
Building codes and standards,123
Building performance rating and reporting,79
Action to to advance net zero carbon municipal buildings,47
"Purchase of low-carbon electricity, heat, steam or cooling (i.e., power purchase agreement, supply agreement, renewable energy credit or other sourcing method)",41
Increase use of clean energy sources for heating and cooling buildings,40
Switching to consumption of low-carbon fuels,37
"Requirements which incentivise net zero carbon, Passivehaus or other ultra-high-efficiency standards for new buildings",28


We can see there are several subcategories of measures in the Stationary Energy sector that relate to buildings, including the most common subcategory relating to energy efficiency in buildings. I will therefore filter these out and re-assign them as a separate category in the steps below.

In [8]:
sectors = ['Stationary energy', 'Generation of grid-supplied energy', 'Transportation', 'Waste']

## Filter emissions reduction actions data for actions:
## - In the above sectors
## - or, even if not in the above sectors, actions related to buildings
## - only actions that are in the implementation or operation phase
mitactions_filtered = city_mitactions[((city_mitactions['Primary emissions sector'].isin(sectors)) | (city_mitactions['Action type'].str.contains('building', case=False)) | (city_mitactions['Action type'].str.contains('buildings', case=False)) |
                                    (city_mitactions['Action description and web link'].str.contains('building', case=False)) | (city_mitactions['Action description and web link'].str.contains('buildings', case=False))) &
                                     ((city_mitactions['Status of action in the reporting year'].str.contains('implementation', case=False)) | 
                                   (city_mitactions['Status of action in the reporting year'].str.contains('operation', case=False)))]

## Drop remaining unnecessary columns
mitactions_filtered.drop(['Estimated annual energy savings (MWh/year)', 'Estimated annual renewable energy generation (MWh/year)'], axis=1, inplace=True)                      

C:\Users\Jones020\AppData\Local\Temp\ipykernel_11928\2825245019.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mitactions_filtered.drop(['Estimated annual energy savings (MWh/year)', 'Estimated annual renewable energy generation (MWh/year)'], axis=1, inplace=True)


In [9]:
## Buildings is listed as a subsector within Stationary Energy
## Create new primary sector definition for Buildings from the rows where Action type contains Buildings; keep other primary sector definitions
mitactions_filtered.reset_index(inplace= True)

for index, row in mitactions_filtered.iterrows():
    # convert the 'Action type' column to a string type
    action_type_str = str(row['Action type'])
    if 'building' in action_type_str.lower():
        mitactions_filtered.at[index, 'URBAN SECTOR'] = 'Buildings'
    else:
        mitactions_filtered.at[index, 'URBAN SECTOR'] = mitactions_filtered.at[index, 'Primary emissions sector']

## Check result
mitactions_filtered

C:\Users\Jones020\AppData\Local\Temp\ipykernel_11928\1274748531.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mitactions_filtered.at[index, 'URBAN SECTOR'] = mitactions_filtered.at[index, 'Primary emissions sector']


,index,Questionnaire,Organization Number,Organization Name,City,Country,Number of times reporting,Primary emissions sector,Action type,Action description and web link,...,Funding source(s),Status of action in the reporting year,Inclusion in climate action plan and/or jurisdiction development / master plan,Total cost of action,Currency,Population,Population Year,City Location,Last update,URBAN SECTOR
0,5,Cities 2022,31172,Mexico City,Mexico City,Mexico,10,Stationary energy,On-site renewable energy generation,Central Solar Fotovoltaica de 18 MW en la Cent...,...,International (including ODA); Jurisdiction's ...,Action in operation (jurisdiction-wide),Action is included in climate action plan and/...,400000000,MXN Mexican Peso,9041395,2018.0,POINT (-99.1332 19.4326),04/24/2023 02:57:02 AM,Stationary energy
1,6,Cities 2022,35880,Municipality of Porto Alegre,Porto Alegre,Brazil,9,Stationary energy,LED / CFL / other luminaire technologies,A PPP da Iluminação Pública é uma concessão ad...,...,Public-private partnerships,Action in operation (across most of jurisdiction),No climate action plan and/or development/mast...,403000000,BRL Brazilian Real,1492530,2021.0,POINT (51 30),04/24/2023 02:57:02 AM,Stationary energy
2,7,Cities 2022,58483,"City of Surrey, BC",NaN,Canada,3,Transportation,"Other, please specify: Strategy that looks at ...","In early 2020, the City began to create a new ...",...,Jurisdiction's own resources,Implementation underway with completion expect...,"Other, please specify: Action included in the ...",NaN,CAD Canadian Dollar,571610,2019.0,NaN,04/24/2023 02:57:02 AM,Transportation
3,8,Cities 2022,848565,Chicago Metropolitan Mayors Caucus,NaN,United States of America,2,Stationary energy,"Purchase of low-carbon electricity, heat, stea...",CEJA authorizes community solar in Illinois. ...,...,Public-private partnerships,Implementation underway with completion expect...,Action is included in climate action plan and/...,NaN,USD US Dollar,8577735,2020.0,NaN,04/24/2023 02:57:02 AM,Stationary energy
4,10,Cities 2022,826396,Municipality of Sintra,Sintra,Portugal,5,Stationary energy,LED / CFL / other luminaire technologies,Promote the replacement of inefficient lightin...,...,"Other, please specify source(s): Owners and/or...",Implementation underway with completion expect...,Action is included in climate action plan and/...,2116451.88,EUR Euro,392887,2021.0,POINT (-9.38811 38.7984),04/24/2023 02:57:02 AM,Stationary energy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2542,5012,Cities 2022,826201,Ayuntamiento de Zapopan,NaN,Mexico,3,Waste,Bans or restrictions on single use or non-recy...,Adecuación de reglamentos municipales para res...,...,Jurisdiction's own resources,Implementation complete in the reporting year,Action is not included in climate action plan ...,NaN,MXN Mexican Peso,1476491,2020.0,NaN,04/24/2023 02:57:02 AM,Waste
2543,5015,Cities 2022,848476,Municipalidad de Cañas,NaN,Costa Rica,3,Waste,Increase awareness/engage public on waste redu...,El alcance de la acción es representativa para...,...,Jurisdiction's own resources,Action in operation (jurisdiction-wide),Action is included in climate action plan and/...,32000000,CRC Costa Rican Colon,26201,2011.0,NaN,04/24/2023 02:57:02 AM,Waste
2544,5021,Cities 2022,74539,"City of Oberlin, OH",NaN,United States of America,2,Generation of grid-supplied energy,Low or zero carbon energy supply generation,Our municipally owned electric power company h...,...,Climate finance (carbon credits); Jurisdiction...,Implementation complete in the reporting year,Action is included in climate action plan and/...,2600000,USD US Dollar,8249,2021.0,NaN,04/24/2023 02:57:02 AM,Generation of grid-supplied energy
2545,5022,Cities 2022,35883,"City of San José, CA","San José, CA",United States of America,10,Stationary energy,Increase use of clean energy sources for heati...,"In June 2018, the City was awarded a $325,000 ...",...,Regional funds and programm

<h3> Pulling 2021 Data for Means of Implementation </h3>

CDP changed their questionnaire between 2022 and 2021. In the 2021 questinnaire, they asked cities to categorize their emissions reduction action from a drop down menu of 'means of implementation'. However, this information is no longer available in the 2022 response data. So I import here the 2021 Emissions Reduction Action dataset and match the information for means of implementation from those measures if they were also reported in 2022.

In [10]:
## 2021 Emissions Reduction Action data from CDP is used to fulfill the following data needs:
## 1) Categorization of some actions by their "means of implementation" (if they were reported by a city in both 2021 and 2022)
## 2) Action data for Den Haag, because not enough useful informationw was provided in reporting on their actions from 2022

mitactions_2021 = pd.read_csv(r''+dir_data+'2021_Cities_Emissions_Reduction_Actions.csv')

In [11]:
citynames_2021 = ['Balikpapan City Government', 'City of Lakewood, CO', 'City of Buenos Aires', 'Gemeente Den Haag']

sectoral_mitactions = mitactions_2021[((mitactions_2021['Mitigation Action'].str.contains('energy', case=False)) | (mitactions_2021['Mitigation Action'].str.contains('buildings', case=False)) |
                         (mitactions_2021['Mitigation Action'].str.contains('waste', case=False)) | (mitactions_2021['Mitigation Action'].str.contains('transport', case=False))) &
                                       ((mitactions_2021['Implementation Status'] == 'Implementation') | (mitactions_2021['Implementation Status'] == 'Operation'))]

# case_mitactions = sectoral_mitactions[(sectoral_mitactions['Organization'].isin(citynames_2021))]
                                       
nodupes_mitactions = sectoral_mitactions[sectoral_mitactions['Action Title'].duplicated(keep='first') == False]

nodupes_mitactions.drop(columns='Access', inplace=True)

C:\Users\Jones020\AppData\Local\Temp\ipykernel_11928\3696083980.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodupes_mitactions.drop(columns='Access', inplace=True)


In [12]:
temp_final = nodupes_mitactions[nodupes_mitactions['Organization'].isin(citynames_2021)]
name_dupes = sectoral_mitactions[(sectoral_mitactions['Organization'].isin(citynames_2021)) & (sectoral_mitactions['Action Title'].duplicated(keep='first') == True)]
name_dupes_grouped = name_dupes.groupby(['Action Title'], as_index=False).agg({'Means of Implementation': ', '.join})
## Found solution to group by with string columns here: https://www.statology.org/pandas-groupby-concatenate-strings/
name_dupes_grouped.rename(columns={'Means of Implementation':'Duplicate Means of Implementation'}, inplace=True)
name_merged = pd.merge(temp_final, name_dupes_grouped, how='left', on=['Action Title'])
## Explanation of join types: https://pandas.pydata.org/docs/user_guide/merging.html
name_merged['Means of Implementation: All'] = name_merged['Means of Implementation'] + ', ' + name_merged['Duplicate Means of Implementation']
name_merged.drop(columns=['Means of Implementation', 'Duplicate Means of Implementation'], inplace=True)

## Check result
name_merged[['Action Title', 'Means of Implementation: All']]

,Action Title,Means of Implementation: All
0,Hacia una Economía Circular,"Development and implementation of action plan,..."
1,Energy-efficient commerce,"Financial mechanism, Infrastructure developmen..."
2,Eficiencia puertas adentro,"Sustainable public procurement, Capacity build..."
3,Energy-efficient housing: Making residential b...,"Policy and regulation, Infrastructure developm..."
4,Low-emission road traffic: Light EV infrastruc...,"Development and implementation of action plan,..."
5,Bicycle and Pedestrian Outreach,"Education, Policy and regulation, Stakeholder ..."
6,"Más bicis, menos emisiones","Assessment and evaluation activities, Infrastr..."
7,Sustainable Energy and Water Resource Center,"Assessment and evaluation activities, Capacity..."
8,Car Free Day,NaN
9,Más y mejor separación en origen,"Awareness raising program or campaign, Verific..."


<h2> Mitigation Plans </h2>

In [13]:
## Check available data
city_mitplans.columns

## No further processing needed

Index(['Reporting Year', 'Organization Number', 'Organization Name', 'City',
       'Country', 'Climate action plan type',
       'Attachment/ link and name of plan',
       'Confirm attachment/ link provided',
       'Boundary of plan relative to jurisdiction boundary',
       'Processes for monitoring, evaluation and updates of plan',
       'Funding sources and financial instruments to finance plan',
       'Stakeholders engaged',
       'Describe if and how climate-related scenarios have informed the plan',
       'Primary author(s) of plan',
       'Assessment of co-benefits, trade-offs and synergies of actions included in the plan',
       'Year of formal approval of plan', 'End year of plan',
       'Total cost of implementation of plan', 'Sectors covered by plan',
       'Population', 'Population Year', 'City Location', 'Last update'],
      dtype='object')

<h2> Emissions Reduction Targets </h2>

In [15]:
## Check available data
city_targets.columns

## No futher processing needed

Index(['Questionnaire', 'Organization Number', 'Organization Name', 'City',
       'Country', 'Number of times reporting', 'Target type',
       'Target boundary', 'Emissions covered in target',
       'Are carbon credits currently used or planned to beÂ used to achieve this target?',
       'Percentage of target to be met using carbon credits',
       'Year target was established',
       'Covered emissions in year target was established (metric tonnes CO2e)',
       'Base year', 'Covered emissions in base year (metric tonnes CO2e)',
       'Emissions intensity figure in base year (metric tonnes CO2e per capita or GDP)',
       'Target year',
       'Estimated business as usual emissions in target year (metric tonnes CO2e)',
       'Percentage of emissions reduction (including offsets and carbon dioxide\nremoval)*^',
       'Net emissions in target year (after offsets and carbon dioxide removal) for base year emissions targets [auto-calculated]',
       'Net emissions in target year (

<h1> National Data: Cleaning & Aggregation </h1>

In [16]:
# country_isos = ['IDN', 'USA', 'ARG', 'NLD', 'EU']

# countries = ['Indonesia','United States', 'Argentina', 'Netherlands', 'EU']

## Import datasets of national emissions for each selected country
cait_files = ['IDN_CAIT','USA_CAIT','ARG_CAIT', 'NLD_CAIT']

cait_frames = []

for i in range(len(cait_files)):
    cait_df = pd.read_csv(r''+dir_data+cait_files[i]+'.csv', on_bad_lines='skip', sep=',')
    ## Only take most relevant columns: emissions sector, unit of measure, most recent emissions years
    cait_new = cait_df.iloc[:-2,:].loc[:,['Sector','unit','2018','2019']]
    ## Add country code column for easier recognition when datasets are concatenated
    cait_new['Country Code'] = cait_files[i]
    cait_frames.append(cait_new)

## Concatenate emissions data for all selected countries to one dataset
cait_full = pd.concat(cait_frames)

## Import dataset of climate change related national policies downloaded from New Climate Institute's Climate Policy Database
nc_preselected = pd.read_csv(r''+dir_data+'ALL_NC_Policies.csv')
## Filter for only policies that are in force
nc_inforce = nc_preselected[nc_preselected['Implementation state'] == 'In force']

## Import World Bank population data
wb_pop = pd.read_excel(r''+dir_data+'WorldBank_Population.xlsx', sheet_name='Data')

## Import dataset of national emissions reduction targets manually gathered for selected countries
natl_targets =  pd.read_excel(r''+dir_data+'National_Targets.xlsx')


<h1> Combined Data Aggregation & Export </h1>

In [18]:
## Set up list with tuples containing each city's official name, country's ISO code and country name
citycountry_pairs = [('City of Buenos Aires', 'ARG', 'Argentina'), ('City of Lakewood, CO', 'USA', 'United States'), ('Balikpapan City Government', 'IDN', 'Indonesia'), 
                     ('Municipality of The Hague', 'NLD', 'Netherlands')]

## Loop over city-country pair list, taking information from each dataset that matches each city, iso and country name, respectively
for city, iso, country in citycountry_pairs: 
        with pd.ExcelWriter(r'TEST_'+city+'_Factsheet.xlsx') as writer:
            city_emissions[city_emissions['Organization Name'] == city].to_excel(writer, sheet_name='City Emissions')
            city_targets[city_targets['Organization Name'] == city].to_excel(writer, sheet_name='City Targets')
            mitactions_filtered[mitactions_filtered['Organization Name'] == city].to_excel(writer, sheet_name='City Mitigation Actions')
            city_mitplans[city_mitplans['Organization Name'] == city].to_excel(writer, sheet_name='City Mitigation Plan')
            cait_full[cait_full['Country Code'].str.contains(iso)].to_excel(writer, sheet_name='National Sectoral Emissions')
            nc_inforce[nc_inforce['Country ISO'] == iso].to_excel(writer, sheet_name='National Sectoral Policies')
            wb_pop[wb_pop['Country Code'] == iso].to_excel(writer, sheet_name='National Population')  
            natl_targets[natl_targets['Country'] == iso].to_excel(writer, sheet_name='National Targets')